In [ ]:
# This notebook was made with the great help of an advance programming peer

In [66]:
import pandas as pd
import os.path as path
import string
import numpy as np
from pathlib import Path

In [70]:
# create a dataframe with all the data from all files

taxColNames = list(string.ascii_lowercase)[:21]

def createEmptyDataFrame(colNames):
    cols = ['abundance', 'sampleRead']
    cols.extend(colNames)
    return pd.DataFrame(columns=cols)

def sampleAndReadFromFilename(fn):
    basename = path.basename(fn)
    return [basename[0:2],basename[2:4]]

def readFileIntoDataFrame(fn, df):
    [sampleid,readid]= sampleAndReadFromFilename(fn)
    with open(fn, 'r') as file:
        data = []
        for line in file.readlines():
            [abundance, tis] = line.strip().split('\t', 1)
            newRow = [int(abundance), sampleid+readid]
            newRow.extend(tis.split('\t'))
            data.append(newRow)
        return df.append(pd.DataFrame(data, columns = df.columns))

def readAllFilesIntoNewDataFrame(dir, colNames):
    df = createEmptyDataFrame(colNames)
    for fn in Path(dir).glob('*.txt'):
        df = readFileIntoDataFrame(str(fn), df)
    return df

allData = readAllFilesIntoNewDataFrame('../2.txt_files', taxColNames)

allData.sampleRead.astype('category')

'done'

'done'

In [80]:
def makeTempDFs(allData):
    allTempDFs = []
    for eachSampleRead in sorted(allData.sampleRead.unique()):
        tempDF = allData[allData.sampleRead == eachSampleRead].copy()
        tempDF[eachSampleRead] = tempDF.abundance
        del tempDF['abundance']
        del tempDF['sampleRead']
        allTempDFs.append(tempDF)
    return allTempDFs

def mergeTempDFs(tempDFs, colNames):
    mergedDFs = pd.DataFrame(columns=colNames)
    for tempDF in tempDFs:
        mergedDFs = mergedDFs.merge(tempDF, how="outer", on=colNames)
    return mergedDFs

results = mergeTempDFs(makeTempDFs(allData), taxColNames)

'done'

'done'

In [84]:
results.to_csv('../7.AllDataTogether_BigTable/allDataTogether_MergedOuter.csv',index = False)